<font size="6">Run the LULUCF part of the AFOLU model</font> 

<font size="4">Must be run using the utilities_and_variables.ipynb kernel</font> 

In [33]:
# 
def calculate_and_upload_LULUCF_fluxes(bounds, is_final):

    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees)

    no_data_val = 255

    
    ### Part 1: Downloads chunks and check for data

    # Dictionary of downloaded layers
    download_dict = {}
    layers = {}

    download_dict = {
        "land_cover_2000": f"{composite_LC_uri}/2000/raw/{tile_id}.tif",
        "land_cover_2005": f"{composite_LC_uri}/2005/raw/{tile_id}.tif",
        "land_cover_2010": f"{composite_LC_uri}/2010/raw/{tile_id}.tif",
        "land_cover_2015": f"{composite_LC_uri}/2015/raw/{tile_id}.tif",
        "land_cover_2020": f"{composite_LC_uri}/2020/raw/{tile_id}.tif",

        "agc_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/aboveground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_AGC_ha_2000.tif",
        "bgc_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/belowground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_BGC_ha_2000.tif",
        # "deadwood_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/deadwood_carbon/extent_2000/standard/20230222/{tile_id}_Mg_deadwood_C_ha_2000.tif",
        # "litter_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/litter_carbon/extent_2000/standard/20230222/{tile_id}_Mg_litter_C_ha_2000.tif",
        # "soil_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/soil_carbon/intermediate_full_extent/standard/20231108/{tile_id}_soil_C_full_extent_2000_Mg_C_ha.tif",

        # "r_s_ratio": f"s3://gfw2-data/climate/carbon_model/BGB_AGB_ratio/processed/20230216/{tile_id}_BGB_AGB_ratio.tif",

        # "drivers": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/{tile_id}_tree_cover_loss_driver_processed.tif",
        # "planted_forest_type": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_type/SDPTv2/20230911/{tile_id}_plantation_type_oilpalm_woodfiber_other.tif",
        # "peat": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif",
        # "ecozone": f"s3://gfw2-data/fao_ecozones/v2000/raster/epsg-4326/10/40000/class/gdal-geotiff/{tile_id}.tif",   # Originally from gfw-data-lake, so it's in 400x400 windows 
        # "iso": f"s3://gfw2-data/gadm_administrative_boundaries/v3.6/raster/epsg-4326/10/40000/adm0/gdal-geotiff/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows
        # "ifl_primary": f"s3://gfw2-data/climate/carbon_model/ifl_primary_merged/processed/20200724/{tile_id}_ifl_2000_primary_2001_merged.tif"
    }

    # for year in range(first_year, last_year+1):
    #     download_dict[f"burned_area_{year}"] = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year}_{tile_id}.tif",
    #     download_dict[f"forest_disturbance_{year}"] = f"s3://gfw2-data/landcover/annual_forest_disturbance/raw/{year}_{tile_id}.tif"  

    # Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
    tile_exists = check_for_tile(download_dict)

    if tile_exists == 0:
        return

    # Note: If running in a local Dask cluster, prints to console may be duplicated. Doesn't happen with a Coiled cluster of the same size (1 worker).
    # Seems to be a problem with local Dask getting overwhelmed by so many futures being created and downloaded from s3. 
    futures = prepare_to_download_chunk(bounds, download_dict, no_data_val)

    # print(futures)

    if not is_final:
        print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
    
    # Waits for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # print(layers)
    
    # Checks chunk for data. Skips the chunk if it has no data in it.
    data_in_chunk = check_chunk_for_data(layers, "land_cover_", bounds_str, tile_id, no_data_val)

    if data_in_chunk == 0:
        return

    
    ### Part 2: Create a separate dictionary for each chunk datatype so that they can be passed to Numba as separate arguments
    ### Note: need to add new code if new input chunk types are added

    # Initializes empty dictionaries for each type
    uint8_dict_layers = {}
    float32_dict_layers = {}
    
    # Iterates through the downloaded chunk dictionary and distributes arrays to a separate dictionary for each data type
    for key, array in layers.items():
        if array.dtype == np.uint8:
            uint8_dict_layers[key] = array
        elif array.dtype == np.float32:
            float32_dict_layers[key] = array

    # Creates numba-compliant typed dict for each type of array
    typed_dict_uint8 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.uint8, 2, 'C')  # Assuming 2D arrays of uint8
    )

    typed_dict_float32 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.float32, 2, 'C')  # Assuming 2D arrays of uint8
    )

    # Populates the numba-compliant typed dicts
    for key, array in uint8_dict_layers.items():
        typed_dict_uint8[key] = array

    for key, array in float32_dict_layers.items():
        typed_dict_float32[key] = array

    
    ### Part 3: Calculates LULUCF fluxes and densities
   
    print(f"Calculating LULUCF fluxes and carbon densities in {bounds_str} in {tile_id}: {timestr()}")

    # agc_out_dict, bgc_out_dict = LULUCF_fluxes(
    #     typed_dict_uint8, typed_dict_float32 
    # )

    # print(agc_out_dict)
    # print(bgc_out_dict)

    out_dict = LULUCF_fluxes(
        typed_dict_uint8, typed_dict_float32 
    )

    print(out_dict)
    
    # # Output files to upload to s3
    # LULUCF_output_dict[f"IPCC_change_{year-5}_{year}"] = [IPCC_change, "uint8", "IPCC_basic_change", f'{year-5}_{year}']  

    # # save_and_upload_small_raster_set(bounds, chunk_length_pixels, tile_id, bounds_str, LULUCF_output_dict, is_final)

    
    # # # Clear memory of unneeded arrays
    # # del LULUCF_output
    # # del LULUCF_output_dict

    return f"Success for {bounds_str}: {timestr()}"

In [69]:
# Function to calculate LULUCF fluxes and carbon densities
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def LULUCF_fluxes(uint8_layers, float32_layers):

    # print("Inside numba")
    # print(uint8_layers)
    # print(uint8_layers["land_cover_2000"])
    # print(uint8_layers["land_cover_2000"].shape)

    # Nested dictionary with all outputs: outer nesting is the output types (AGC density, BGC removals, etc.), inner nesting is the year (2000-2005, 2005-2010, etc.)
    out_dict_all_data = {}
            
    # Output dictionaries for specific output types
    state_out_dict = {}
    agc_dens_out_dict = {}
    bgc_dens_out_dict = {}
    agc_flux_out_dict = {}
    bgc_flux_out_dict = {}

    end_years = list(range(first_year, last_year+1, 5))[1:]

    for year in end_years:
                
        # print(year)
        state_out = np.zeros(uint8_layers["land_cover_2000"].shape)
        state_out = state_out.astype('uint16')
        agc_dens_out = np.zeros(float32_layers["agc_2000"].shape)
        bgc_dens_out = np.zeros(float32_layers["agc_2000"].shape)
        agc_flux_out = np.zeros(float32_layers["agc_2000"].shape)
        bgc_flux_out = np.zeros(float32_layers["agc_2000"].shape)

        IPCC_previous_block = uint8_layers[f"land_cover_{year-5}"].astype('uint16')  # Recasting uint8 as uint16 here works for both jit and Python modes
        IPCC_current_block = uint8_layers[f"land_cover_{year}"].astype('uint16')  # Recasting uint8 as uint16 here works for both jit and Python modes

        # Iterates through all pixels in the chunk
        for row in range(IPCC_current_block.shape[0]):
            for col in range(IPCC_current_block.shape[1]):
                
                IPCC_previous = IPCC_previous_block[row, col]
                IPCC_current = IPCC_current_block[row, col]

                state_out[row, col] = IPCC_previous + IPCC_current
    
                # # When land cover chunks have "no data"
                # if (IPCC_previous == 0) and (IPCC_current == 0):
                #     IPCC_change_block[row, col] = 0
    
                # else:
                #     # Equation to calculate the IPCC change code
                #     IPCC_change_block[row, col] = ((IPCC_previous - 1) * IPCC_class_max_val) + IPCC_current

        state_out_dict[year] = state_out
        agc_dens_out_dict[year] = agc_dens_out
        bgc_dens_out_dict[year] = bgc_dens_out
        agc_flux_out_dict[year] = agc_flux_out
        bgc_flux_out_dict[year] = bgc_flux_out


    out_dict_all_data["state"] = state_out_dict
    # out_dict_all_data["agc_dens"] = agc_dens_out_dict
    # out_dict_all_data["bgc_dens"] = bgc_dens_out_dict
    # out_dict_all_data["agc_flux"] = agc_flux_out_dict
    # out_dict_all_data["bgc_flux"] = bgc_flux_out_dict

    return out_dict_all_data

In [70]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks
# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)

# # Range of no-data cases for testing
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)


# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 30:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(calculate_and_upload_LULUCF_fluxes)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

Processing 1 chunks
Tile id 50N_010E exists. Proceeding.
Requesting data in chunk 10_50_10_50 in 50N_010E: 20240409_14_36_08
Waiting for requests for data in chunk 10_50_10_50 in 50N_010E: 20240409_14_36_11
Data in chunk 10_50_10_50. Proceeding.
Calculating LULUCF fluxes and carbon densities in 10_50_10_50 in 50N_010E: 20240409_14_36_11
{state: {2005: [[488 488 488 ... 500 500 500]
 [488 488 488 ... 500 500 500]
 [488 488 488 ... 500 500 500]
 ...
 [ 91  87  91 ... 488 488 488]
 [ 84  87  91 ... 488 488 488]
 [ 88  90  92 ... 488 488 488]], 2010: [[488 488 488 ... 500 500 500]
 [488 488 488 ... 500 500 500]
 [488 488 488 ... 500 500 500]
 ...
 [ 88  84  88 ... 488 488 488]
 [ 84  88  88 ... 488 488 488]
 [ 87  90  92 ... 488 488 488]], 2015: [[488 488 488 ... 500 500 500]
 [488 488 488 ... 500 500 500]
 [488 494 488 ... 500 500 500]
 ...
 [ 85  88  90 ... 488 488 488]
 [ 84  87  88 ... 488 488 488]
 [ 87  88  91 ... 488 488 488]], 2020: [[488 488 488 ... 500 500 500]
 [488 488 488 ... 

('Success for 10_50_10_50: 20240409_14_36_12',)

In [ ]:
# Execute without Dask
calculate_and_upload_LULUCF_fluxes([10, 49.75, 10.25, 50], is_final=False)